In [13]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import torch
import torchvision
import math
from sklearn.preprocessing import StandardScaler
from thermography_dataset_one_layer import ThermDataset
from torch.utils.data import DataLoader
import seaborn as sns

In [14]:
dataset = ThermDataset(fp='combined_data.xlsx', noise_scale=0, layer=False)
dataset1 = ThermDataset(fp='combined_data.xlsx', noise_scale=0, layer=1)

In [15]:
dataloader = DataLoader(dataset, shuffle=False, batch_size=32)
dataloader1 = DataLoader(dataset1, shuffle=False, batch_size=32)

In [30]:
for xbatch, ybatch in dataloader:
    print(xbatch[:,0].reshape(-1,1))

tensor([[323.5428],
        [302.3973],
        [342.2532],
        [321.5229],
        [345.2725],
        [370.0598],
        [350.1934],
        [296.7373],
        [342.4752],
        [312.6007],
        [285.5332],
        [286.4303],
        [328.5738],
        [290.0432],
        [289.1134],
        [318.2592],
        [338.3690],
        [319.3260],
        [368.3457],
        [314.5093],
        [317.0036],
        [354.6140],
        [336.1189],
        [345.5775],
        [281.7500],
        [353.0559],
        [353.4449],
        [351.3737],
        [366.9661],
        [302.5507],
        [344.9570],
        [345.6104]])
tensor([[295.4277],
        [291.2227],
        [306.8956],
        [354.1602],
        [285.4774],
        [306.0424],
        [368.4174],
        [307.8785],
        [309.6157],
        [339.7120],
        [313.3491],
        [312.9020],
        [292.5477],
        [344.5678],
        [320.5573],
        [301.6620],
        [337.6302],
        [293.6776],

In [28]:
for xbatch, ybatch in dataloader1:
    print(xbatch)

tensor([[323.5428],
        [302.3973],
        [342.2532],
        [321.5229],
        [345.2725],
        [370.0598],
        [350.1934],
        [296.7373],
        [342.4752],
        [312.6007],
        [285.5332],
        [286.4303],
        [328.5738],
        [290.0432],
        [289.1134],
        [318.2592],
        [338.3690],
        [319.3260],
        [368.3457],
        [314.5093],
        [317.0036],
        [354.6140],
        [336.1189],
        [345.5775],
        [281.7500],
        [353.0559],
        [353.4449],
        [351.3737],
        [366.9661],
        [302.5507],
        [344.9570],
        [345.6104]])
tensor([[295.4277],
        [291.2227],
        [306.8956],
        [354.1602],
        [285.4774],
        [306.0424],
        [368.4174],
        [307.8785],
        [309.6157],
        [339.7120],
        [313.3491],
        [312.9020],
        [292.5477],
        [344.5678],
        [320.5573],
        [301.6620],
        [337.6302],
        [293.6776],